Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

# Continue to clean and explore your data. Make exploratory visualizations.

In [31]:
#See LS_DS_232_assignment
df = pd.read_csv('/content/drive/My Drive/Lambda/Unit 2/Basketball Data/eng_shots_log.csv')
df = df.drop(columns='Unnamed: 0')
df.head(1)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,2229


#New feature: previous shot made

In [53]:
df = df.drop(columns=['PREV_MADE'])
df['PREV_SHOT'] = 'first_shot'
df.head(15) 

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME,PREV_SHOT
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,2229,first_shot
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,brian roberts,203148,0,14,2174,first_shot
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,brian roberts,203148,0,0,2160,first_shot
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,brian roberts,203148,11,47,2147,first_shot
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,brian roberts,203148,10,34,2074,first_shot
5,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,6,2,8:15,9.1,2,4.4,18.4,2,missed,"Williams, Deron",101114,2.6,brian roberts,203148,8,15,1935,first_shot
6,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,7,4,10:15,14.5,11,9.0,20.7,2,missed,"Jack, Jarrett",101127,6.1,brian roberts,203148,10,15,615,first_shot
7,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,8,4,8:00,3.4,3,2.5,3.5,2,made,"Plumlee, Mason",203486,2.1,brian roberts,203148,8,0,480,first_shot
8,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,9,4,5:14,12.4,0,0.8,24.6,3,missed,"Morris, Darius",202721,7.3,brian roberts,203148,5,14,314,first_shot
9,21400890,"MAR 03, 2015 - CHA vs. LAL",H,W,1,1,2,11:32,17.4,0,1.1,22.4,3,missed,"Ellington, Wayne",201961,19.8,brian roberts,203148,11,32,2132,first_shot


In [0]:
trial = df.head(15)

In [65]:
player1 = ''
player2 = ''

for index, x in enumerate(trial['SHOT_RESULT']):
    #account for the first observation 
  if (index == 0):
    trial.loc[index, 'PREV_SHOT'] = 'first_shot'
    player2 = trial.loc[index, 'player_name']
    print('yep')

  if (index >= 1):
      player1 = trial.loc[index, 'player_name']
      if player1 == player2:
        #check to see if this shot was in the same game as the previous
        prev_game = trial.loc[(index-1), 'GAME_ID']
        this_game = trial.loc[index, 'GAME_ID']

        if (prev_game == this_game):
          #set the prev_shot to the player's previous shot
          trial.loc[index, 'PREV_SHOT'] = trial.loc[(index-1), 'SHOT_RESULT']
        else:
          #set the prev_shot to 'first_shot'
          trial.loc[index, 'PREV_SHOT'] = 'first_shot'
      


trial.head(15)

yep


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME,PREV_SHOT
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,2229,first_shot
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,brian roberts,203148,0,14,2174,made
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,brian roberts,203148,0,0,2160,missed
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,brian roberts,203148,11,47,2147,missed
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,brian roberts,203148,10,34,2074,missed
5,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,6,2,8:15,9.1,2,4.4,18.4,2,missed,"Williams, Deron",101114,2.6,brian roberts,203148,8,15,1935,missed
6,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,7,4,10:15,14.5,11,9.0,20.7,2,missed,"Jack, Jarrett",101127,6.1,brian roberts,203148,10,15,615,missed
7,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,8,4,8:00,3.4,3,2.5,3.5,2,made,"Plumlee, Mason",203486,2.1,brian roberts,203148,8,0,480,missed
8,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,9,4,5:14,12.4,0,0.8,24.6,3,missed,"Morris, Darius",202721,7.3,brian roberts,203148,5,14,314,made
9,21400890,"MAR 03, 2015 - CHA vs. LAL",H,W,1,1,2,11:32,17.4,0,1.1,22.4,3,missed,"Ellington, Wayne",201961,19.8,brian roberts,203148,11,32,2132,first_shot


In [70]:
#now do it on the df
player1 = ''
player2 = ''

for index, x in enumerate(df['SHOT_RESULT']):
    #account for the first observation 
  if (index == 0):
    df.loc[index, 'PREV_SHOT'] = 'first_shot'
    player2 = df.loc[index, 'player_name']
    print('yep')

  if (index >= 1):
      player1 = df.loc[index, 'player_name']
      if player1 == player2:
        #check to see if this shot was in the same game as the previous
        prev_game = df.loc[(index-1), 'GAME_ID']
        this_game = df.loc[index, 'GAME_ID']

        if (prev_game == this_game):
          #set the prev_shot to the player's previous shot
          df.loc[index, 'PREV_SHOT'] = df.loc[(index-1), 'SHOT_RESULT']
          #set player2 to the current player
          player2 = df.loc[index, 'player_name']
        else:
          #set the prev_shot to 'first_shot'
          df.loc[index, 'PREV_SHOT'] = 'first_shot'
          print('first')
      else:
        df.loc[index, 'PREV_SHOT'] = 'first_shot'
        player2 = df.loc[index, 'player_name']
        print('new player')
      

Streaming output truncated to the last 5000 lines.
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
new player
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
new player
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
new player
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first
new player
first
first
first
first
first
first
first
first
first
first
first
first
first
first
first

In [88]:
df.tail(20)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME,PREV_SHOT
128049,21400060,"NOV 05, 2014 - BKN vs. MIN",H,L,-7,7,4,7:27,11.0,2,2.2,9.4,2,made,"Pekovic, Nikola",201593,6.0,jarrett jack,101127,7,27,447,made
128050,21400060,"NOV 05, 2014 - BKN vs. MIN",H,L,-7,8,4,6:41,22.7,1,2.0,2.3,2,made,"Williams, Mo",2590,7.7,jarrett jack,101127,6,41,401,made
128051,21400044,"NOV 03, 2014 - BKN vs. OKC",H,W,31,2,1,0:16,NaN,3,3.7,2.6,2,made,"Telfair, Sebastian",2742,5.4,jarrett jack,101127,0,16,2176,first_shot
128052,21400044,"NOV 03, 2014 - BKN vs. OKC",H,W,31,3,2,10:29,19.3,2,3.2,18.9,2,missed,"Telfair, Sebastian",2742,3.8,jarrett jack,101127,10,29,2069,made
128053,21400044,"NOV 03, 2014 - BKN vs. OKC",H,W,31,4,3,6:15,3.2,5,5.1,13.7,2,missed,"Adams, Steven",203500,2.8,jarrett jack,101127,6,15,1095,missed
128054,21400044,"NOV 03, 2014 - BKN vs. OKC",H,W,31,5,3,3:17,4.1,2,2.1,11.1,2,missed,"Telfair, Sebastian",2742,4.2,jarrett jack,101127,3,17,917,missed
128055,21400044,"NOV 03, 2014 - BKN vs. OKC",H,W,31,6,4,11:22,13.8,0,0.6,5.1,2,missed,"Jackson, Reggie",202704,2.1,jarrett jack,101127,11,22,682,missed
128056,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,1,1,1:50,7.1,2,2.3,15.5,2,missed,"Jennings, Brandon",201943,3.1,jarrett jack,101127,1,50,2270,first_shot
128057,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,2,2,10:02,7.3,0,1.0,22.1,3,missed,"Augustin, D.J.",201571,2.9,jarrett jack,101127,10,2,2042,missed
128058,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,3,4,10:57,14.3,2,2.0,18.2,2,made,"Augustin, D.J.",201571,1.1,jarrett jack,101127,10,57,657,missed


# Fit a model. Does it beat your baseline?

In [0]:
#Again, see LS_DS_232_assignment:
#baseline accuracy: 54.8%
#model 1 accuracy: 60.5%
#model 2 accuracy: 60.9%


In [73]:
target = 'SHOT_RESULT'
y = df[target]
y.value_counts(normalize=True)

missed    0.547861
made      0.452139
Name: SHOT_RESULT, dtype: float64

In [74]:
#Can you make a fast, first model that beats guessing?
import numpy as np
from sklearn.model_selection import train_test_split

train = df

# Split df into into train & test sets
train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, test.shape

((102455, 23), (25614, 23))

In [75]:
#Split df into train and validation sets
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, val.shape

((81964, 23), (20491, 23))

In [0]:
# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [78]:
#install category encoders
!pip install category_encoders

In [79]:
#Fit a model using a pipeline

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.one_hot.OneHotEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=100, n_jobs= -1, random_state=42)
)

pipeline.fit(X_train, y_train)
print(f'Valdiation accuracy: {pipeline.score(X_val, y_val)}')

Valdiation accuracy: 0.6084134498072324


# Get your model's permutation importances

In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5

import eli5
from eli5.sklearn import PermutationImportance


In [81]:
transformers = make_pipeline(
    ce.ordinal.OrdinalEncoder(),
    SimpleImputer()
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [82]:
permuter = PermutationImportance(
    model,
    scoring = 'accuracy',
    n_iter=5,
    random_state=42
)

permuter.fit(X_val_transformed, y_val)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fr

In [83]:
feature_names = X_val.columns.to_list()

pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)

SHOT_DIST                     0.074394
CLOSE_DEF_DIST                0.015197
TOUCH_TIME                    0.013918
PTS_TYPE                      0.004578
SHOT_CLOCK                    0.004207
MINUTES                       0.003826
FINAL_MARGIN                  0.003377
player_id                     0.002938
SECONDS_IN_GAME               0.001991
W                             0.001601
LOCATION                      0.001386
GAME_ID                       0.001308
SHOT_NUMBER                   0.000791
CLOSEST_DEFENDER_PLAYER_ID    0.000703
PREV_SHOT                     0.000381
PERIOD                        0.000303
DRIBBLES                      0.000273
SECONDS                       0.000166
dtype: float64

In [84]:
permuter.feature_importances_std_

array([0.00205372, 0.0016836 , 0.00080569, 0.00058187, 0.00166339,
       0.0010099 , 0.00224349, 0.0008568 , 0.00134126, 0.00136736,
       0.0016427 , 0.00054115, 0.00130716, 0.00157369, 0.00114758,
       0.00056963, 0.00117951, 0.00118395])

In [85]:
eli5.show_weights(permuter,
                  top=None, 
                  feature_names = feature_names)

Weight,Feature
0.0744 ± 0.0017,SHOT_DIST
0.0152 ± 0.0033,CLOSE_DEF_DIST
0.0139 ± 0.0045,TOUCH_TIME
0.0046 ± 0.0027,PTS_TYPE
0.0042 ± 0.0033,SHOT_CLOCK
0.0038 ± 0.0026,MINUTES
0.0034 ± 0.0034,FINAL_MARGIN
0.0029 ± 0.0011,player_id
0.0020 ± 0.0023,SECONDS_IN_GAME
0.0016 ± 0.0024,W


# Try xgboost.

In [86]:
from xgboost import XGBClassifier

pipeline = make_pipeline(
    ce.ordinal.OrdinalEncoder(),
    XGBClassifier(n_estimators= 100, random_state = 42, n_jobs = -1)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['LOCATION', 'W', 'PREV_SHOT'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'LOCATION',
                                          'data_type': dtype('O'),
                                          'mapping': A      1
H      2
NaN   -2
dtype: int64},
                                         {'col': 'W', 'data_type': dtype('O'),
                                          'mapping': W      1
L      2
NaN   -2
dtype: int64},
                                         {'col': 'PREV_SHOT',
                                          'data_type': dtype('O'...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.

In [87]:
from sklearn.metrics import accuracy_score
y_pred = pipeline.predict(X_val)
print(f'Val accuracy score: {accuracy_score(y_val, y_pred)}')

Val accuracy score: 0.62207798545703
